<a href="https://colab.research.google.com/github/CarolineLaure/One_Qubit_TensorFlow_example/blob/master/Copy_of_Caroline_Onequbit_Optimal_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

2.2.0-rc1


In [0]:
#Deleseleuc parameters

delta1=2*np.pi*560*10**6
sigma_total=2*np.pi*13*10**3
delta2=-delta1+sigma_total
Omega1=2*np.pi*60*10**6
Omega2=2*np.pi*36*10**6

class Propagator:
    def __init__(self, no_of_steps, delta_t, dim):
        
        self.dim=3
        self.delta_t = delta_t
        self.x = tf.constant(
            [[0., -(Omega1/2.), 0], [-(Omega1/2.), -(delta1/1.),  -(Omega1/2.)], [0., -(Omega2/2.), -(sigma_total/1.)]], dtype=tf.complex128
        )
        self.y = tf.constant(
            [[0 + 0j, 0 - 1j, 0], [0 + 1j, 0 + 0j, 0], [0, 0, 0]], dtype=tf.complex128
        )

        self.generators =  tf.stack([self.x, self.y])
        self.ctrl_amplitudes = tf.Variable(
            tf.zeros([no_of_steps, 2], dtype=tf.float64), dtype=tf.float64
        )

        """
            self.contraction_array determines the neccessity for the extra
            matrix multiplication step in the recursive method self.propagate()
            when the intermediate computation array has length not divisible
            by 2
        """
        self.contraction_array = []
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps = no_of_steps
        for i in range(contraction_array_length):
            self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2)))
            temp_no_of_steps = np.floor(temp_no_of_steps / 2)
  
    """
        exponentials() computes a vector matrix exponential after multiplying
        each self.ctrl_amplitudes row with a the vector of matrices in
        self.generators
    """
    def exponentials(self):
        regularized_amplitudes = 1 / np.sqrt(2) * tf.math.tanh(
            self.ctrl_amplitudes
        )

        exponents = tf.linalg.tensordot(
            tf.cast(regularized_amplitudes, dtype=tf.complex128),
            -2 * np.pi *(0 + 1j) * self.delta_t * self.generators, 1
        )
        return tf.linalg.expm(exponents)
    
    """
        propagate  computes the final propagator by recursively multiplying
        each odd element in the list of matrices with each even element --
        if the length of the array is not divisible by 2 an extra computation
        step is added
    """
    def propagate(self):
        step_exps = self.exponentials()
        for is_odd in self.contraction_array:
            if is_odd:
                odd_exp = step_exps[-1, :, :]
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0:-1:2, :, :]
                )
                step_exps = tf.concat([
                    step_exps[0:-1, :, :],
                    [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]
                ], 0)
            else:
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0::2, :, :]
                )
        return tf.squeeze(step_exps)

    """
        __call__ computes the final propagator fidelity squared with the
        identity operator
    """
    @tf.function
    def infidelity(self):
        propagator = self.propagate()
        tr = tf.linalg.trace(tf.linalg.matmul(self.x, propagator))
        return 1 - tf.math.real(tr * tf.math.conj(tr)) / (3 ** 2)

propagator = Propagator(1000, 0.001, 3)

optimizer = tf.keras.optimizers.Adam(0.01)

propagator.ctrl_amplitudes.assign(
    tf.random.uniform([1000, 2], -1, 1, dtype=tf.float64)
)

def optimization_step():
    with tf.GradientTape() as tape:
        infidelity = propagator.infidelity()
    gradients = tape.gradient(infidelity, [propagator.ctrl_amplitudes])
    optimizer.apply_gradients(zip(gradients, [propagator.ctrl_amplitudes]))
    return infidelity

steps = range(100)
for step in steps:
    current_infidelity = optimization_step()
    print('step %2d: infidelity=%2.5f' %
          (step, current_infidelity))
    
propagator.ctrl_amplitudes.numpy()
#propagator.delta_t
#propagator.no_of_steps

ValueError: ignored